# Plan of Attack

In order to try and improve this model, there are a few areas that we want to investigate:
- Changes to existing modeling approach
- Changes to existing feature preprocessing
- Feature engineering to add new (potentially better) features

In [50]:
from typing import List

import numpy as np
import pandas as pd

from sklearn import model_selection
from sklearn import pipeline, preprocessing, base

from housing.config import DEMOGRAPHICS_PATH, SALES_PATH
from housing.modeling.create_model import load_data

# EDA

Just some initial exploration of the input dataset to make sure I understand some of it's basic properties.

In [51]:
all_cols = [
    "price",
    "bedrooms",
    "bathrooms",
    "sqft_living",
    "sqft_lot",
    "floors",
    "waterfront",
    "view",
    "condition",
    "grade",
    "sqft_above",
    "sqft_basement",
    "yr_built",
    "yr_renovated",
    "zipcode",
    "lat",
    "long",
    "sqft_living15",
    "sqft_lot15",
]

x, y = load_data(SALES_PATH, DEMOGRAPHICS_PATH, all_cols)

x.shape, y.shape

((21613, 43), (21613,))

- Assuming a default holdout set of 25%, this gives us 16,210 training samples and 5,403 holdout samples.
- 43 columns is quite a few, indicating that simple linear models might struggle a bit due to the high dimensionality of the problem.

# Helpers

Utility functions to train a specified model on the specified feature columns.

In [52]:
RANDOM_STATE = 42

def build_pipeline(estimator: base.BaseEstimator) -> pipeline.Pipeline:
    model = pipeline.make_pipeline(
        preprocessing.RobustScaler(),
        estimator
    )
    return model

def train_and_evaluate(model: pipeline.Pipeline, cols: List[str]):
    x, y = load_data(SALES_PATH, DEMOGRAPHICS_PATH, cols)
    x_train, x_test, y_train, y_test = model_selection.train_test_split(
        x,
        y,
        random_state=RANDOM_STATE
    )

    model.fit(x_train, y_train)

    train_score = model.score(x_train, y_train)
    test_score = model.score(x_test, y_test)

    print(f"Train score: {train_score:.4f}")
    print(f"Test score: {test_score:.4f}")

# First step: Recreate provided training method

Standard training copied from provided `create_model.py` to see what our existing production model performance is.

In [53]:
from sklearn import neighbors

cols = [
    'price',
    'bedrooms',
    'bathrooms',
    'sqft_living',
    'sqft_lot',
    'floors',
    'sqft_above',
    'sqft_basement',
    'zipcode'
]

model = build_pipeline(neighbors.KNeighborsRegressor())

train_and_evaluate(model, cols)

Train score: 0.8414
Test score: 0.7281


# Baseline Linear Regression

Linear regression is probably too simplistic for this sort of problem, but it has the advantage of being _interpretable_, which can often be quite helpful when trying to understand a dataset (or present it to a client in a way that is understandable by them). Let's start there to at least get a good baseline for our model performance.

In [54]:
from sklearn import linear_model

model = build_pipeline(linear_model.LinearRegression())

train_and_evaluate(model, cols)

Train score: 0.7319
Test score: 0.7190


The easiest thing to do to try and improve the model is provide more columns of data, to see if more information improves the performance.

In [ ]:
all_cols = [
    "price",
    "bedrooms",
    "bathrooms",
    "sqft_living",
    "sqft_lot",
    "floors",
    "waterfront",
    "view",
    "condition",
    "grade",
    "sqft_above",
    "sqft_basement",
    "yr_built",
    "yr_renovated",
    "zipcode",
    "lat",
    "long",
    "sqft_living15",
    "sqft_lot15",
]

model = build_pipeline(linear_model.LinearRegression())

train_and_evaluate(model, all_cols)

Train score: 0.7939
Test score: 0.7890


That's already better, but we can probably continue to improve.

# An Obvious Next Step: A Regularized Linear Model

Adding regularization to an existing linear model can sometimes be an easy way to improve model performance, typically by preventing overfitting by penalizing the model for large coefficients.

In [64]:
model = build_pipeline(linear_model.Ridge(alpha=1.0))

train_and_evaluate(model, all_cols)

Train score: 0.7932
Test score: 0.7884


Not much difference from the simple linear model - this would indicate to me that we should try a wholly different approach than the standard linear regression.

# A More Complex Modeling Approach

Since we have a lot of features, especially after joining in demographic data, I often find it's best to take and ensemble approach that allows the model to determine the most important features. Random forests in particular allow for the handling of nonlinear relationships between features and targets, while still remaining fairly robust to over-fitting due to the ensemble nature.

In [58]:
from sklearn import ensemble

model = build_pipeline(
    ensemble.RandomForestRegressor(n_estimators=100, random_state=RANDOM_STATE)
)

train_and_evaluate(model, all_cols)

Train score: 0.9841
Test score: 0.8774


Much better! We improved by ~9 percentage points over our initial baseline. Though we are very clearly overfitting to the training data, since our performance on the holdout set is over 10 percentage points lower. This could be due to inherent differences in the training/testing split (i.e. we have some bias and/or differences between the two datasets, rather than being similar distributions). To test this hypothesis, let's try some k-fold cross-validation (which should hopefully eliminate discrepancies in the training/test set) to see if that improves our test score at all.

In [59]:
def train_and_evaluate_with_cross_validation(model: pipeline.Pipeline, cols: List[str], cv: int = 5):
    x, y = load_data(SALES_PATH, DEMOGRAPHICS_PATH, cols)

    cv_results = model_selection.cross_validate(
        model,
        x,
        y,
        cv=cv,
        return_train_score=True,
        n_jobs=-1
    )

    print(f"Average train score: {cv_results['train_score'].mean():.4f}")
    print(f"Average test score: {cv_results['test_score'].mean():.4f}")

model = build_pipeline(
    ensemble.RandomForestRegressor(n_estimators=100, random_state=RANDOM_STATE)
)

train_and_evaluate_with_cross_validation(model, all_cols, cv=5)

Average train score: 0.9832
Average test score: 0.8844


Only very slightly better - it appears that our `RandomForestRegressor` is responsible for the over-fitting.

<br>

If I were to continue to diagnose this over-fitting (which can be quite time-consuming, so I'm not actually going to), I would set up a grid search to do some hyperparameter tuning on the `RandomForestRegressor`, probably targeting a few key hyperparameters like `n_estimators`, `max_depth`, `max_features`, `min_samples_split`, and `min_samples_leaf`. I could even set up a custom metric to search for the hyperparameters that minimize the difference between the train and test scores, just to try and find the parameters that minimize overfitting. This may not necessarily be the "best" model (as measured by performance on the holdout set), but it would still be informative to understand the root cause of overfitting.


# Feature Engineering

Despite the overfitting, I'm still pretty happy with the performance of the model on holdout data. To see if we can continue to increase performance though, let's try engineering some additional features and using the same `RandomForestRegressor` model as above.

In [ ]:
data = pd.read_csv(
    SALES_PATH,
    dtype={'zipcode': str}
)
demographics = pd.read_csv(
    DEMOGRAPHICS_PATH,
    dtype={'zipcode': str}
)

data = data.merge(
    demographics,
    on="zipcode",
    how="left"
)

data["date"] = data["date"].astype("datetime64[ns]")

# Some simple boolean flags to better capture common house features
data["is_renovated"] = (data["yr_renovated"] > 0).astype(int) 
data["has_basement"] = (data["sqft_basement"] > 0).astype(int)
# Some other possibly useful real-estate features
data["total_rooms"] = data["bedrooms"] + data["bathrooms"]
data["living_to_lot_ratio"] = data["sqft_living"] / data["sqft_lot"]

# Price per sqft is a common real-estate metric, but including it here enables the model to directly learn the price due
# to the combination of this feature and sqft_living, rather than having to infer it indirectly.
# data["price_per_sqft"] = data["price"] / data["sqft_living"]


# Some time/seasonal features
data["age"] = data["date"].dt.year - data["yr_built"]
data["renovated_age"] = np.where(
    data["yr_renovated"] > 0,
    data["date"].dt.year - data["yr_renovated"],
    data["age"]
)
data["month_sold"] = data["date"].dt.month

data["mean_price_zipcode"] = data.groupby("zipcode")["price"].transform("mean")

# Offset by a month to better align with "seasons"
# (i.e. winter = Dec, Jan, Feb, spring = Mar, Apr, May, etc)
data["season_sold"] = (data["month_sold"] % 12 // 3) + 1  # 1 = Winter, 2 = Spring, 3 = Summer, 4 = Fall

y = data.pop("price")
x = data.drop(columns=["date", "id"])


x_train, x_test, y_train, y_test = model_selection.train_test_split(
    x,
    y,
    random_state=RANDOM_STATE
)

model = build_pipeline(
    ensemble.RandomForestRegressor(n_estimators=100, random_state=RANDOM_STATE)
)

model.fit(x_train, y_train)

train_score = model.score(x_train, y_train)
test_score = model.score(x_test, y_test)

print(f"Train score: {train_score:.4f}")
print(f"Test score: {test_score:.4f}")

Train score: 0.9843
Test score: 0.8759


Not much better - I interpret this result as the `RandomForestRegressor` already doing a good job of picking out the sorts of non-linearities that these additional features describe just using the base set of columns.

# Feature Preprocessing

The `RobustScaler` is typically pretty good at properly scaling features to improve model training, but there are a few other common options available in `scikit-learn`:
- `RobustScaler`: Scales relative to interquartile range (IQR), to better avoid outliers
- `StandardScaler`: Translates features into z-space (i.e. std. devs. relative to feature mean)
- `MinMaxScaler`: Linearly scales the features in the range (min, max)

In [61]:
robust_scaler_model = pipeline.make_pipeline(
    preprocessing.RobustScaler(),
    ensemble.RandomForestRegressor(n_estimators=100, random_state=RANDOM_STATE)
)

robust_scaler_model.fit(x_train, y_train)
train_score = robust_scaler_model.score(x_train, y_train)
test_score = robust_scaler_model.score(x_test, y_test)

print("RobustScaler Model Results:")
print(f"Train score: {train_score:.4f}")
print(f"Test score: {test_score:.4f}")

RobustScaler Model Results:
Train score: 0.9843
Test score: 0.8759


In [62]:
standard_scaler_model = pipeline.make_pipeline(
    preprocessing.StandardScaler(),
    ensemble.RandomForestRegressor(n_estimators=100, random_state=RANDOM_STATE)
)
standard_scaler_model.fit(x_train, y_train)
train_score = standard_scaler_model.score(x_train, y_train)
test_score = standard_scaler_model.score(x_test, y_test)

print("StandardScaler Model Results:")
print(f"Train score: {train_score:.4f}")
print(f"Test score: {test_score:.4f}")

StandardScaler Model Results:
Train score: 0.9843
Test score: 0.8760


In [63]:
minmax_scaler_model = pipeline.make_pipeline(
    preprocessing.MinMaxScaler(),
    ensemble.RandomForestRegressor(n_estimators=100, random_state=RANDOM_STATE)
)
minmax_scaler_model.fit(x_train, y_train)
train_score = minmax_scaler_model.score(x_train, y_train)
test_score = minmax_scaler_model.score(x_test, y_test) 

print("MinMaxScaler Model Results:")
print(f"Train score: {train_score:.4f}")
print(f"Test score: {test_score:.4f}")

MinMaxScaler Model Results:
Train score: 0.9843
Test score: 0.8759


Again, not much change here, indicating that the specifics of the feature pre-processing aren't really driving the modeling performance here.

# Conclusions

We've done a pretty quick and basic investigation into some different modeling approaches here. Below is a summary of our findings:
- Using a simple linear regression model, we got a baseline performance of R^2 = 0.719.
- This is similar to the performance of the current production model (R^2 = 0.728).
- We were able to improve this linear regression (R^2 = 0.789) simply by providing more columns to the model - more features gives the model more information.
- Adding in some regularization to this linear model doesn't improve performance much.
- Switching to a more complex model (a random forest regressor) gave us a big jump in performance (R^2 = 0.877), due to random forest's ability to determine the most important features.
- There appears to be some overfitting which wasn't resolved by introducing some cross-validation
- Additional feature engineering and/or different feature preprocessing doesn't seem to make much of a difference, indicating that the random forest is already picking up on the non-linear relationships between the features and the target home price.

# Next Steps/Future Work

All of the above work was done with the existing dataset in mind - a larger project focused on improving the client's model would almost certainly involve investigating additional datasets that could enrich the current features and improve the model's ability to determine home prices. To that end, I would start by attempting to source the following additional datasets, ordered by most-useful to least-useful:
- Additional housing sales data for this same region spanning different years: the current dataset only spans a small ~1-year time range, so our ability to generalize beyond this time frame is limited. Housing markets often have large-scale fluctuations over time, so getting a larger time-range of data would let us better pick up on both seasonal and multi-year trends in home prices.
- Additional housing sales data from other regions: while housing markets do differ regionally, there are (hopefully) general trends that apply across markets, so having additional data spanning different markets (and appropriately encoding that spatial information into the model) would still allow us to build a model of home prices that generalizes well across regions.
- More granular demographic information: Zipcode-level demographics are clearly useful for this model, but FIPS- or census-tract-level demographics could allow the model to pick up even finer-grained spatial information in the housing market.